In [15]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    import os
    os.chdir('/content')
    if not os.path.isdir('/content/EVA-2-Group/'):
        !git clone https://github.com/sambitdash/EVA-2-Group.git
    os.chdir('/content/EVA-2-Group/Session-19')
    !pwd
    
    !git config user.email "sambitdash@gmail.com"
    !git config user.name "Sambit Kumar Dash"
    !git config user.password "your password"
    !git status

In [16]:
import cv2
import os
import numpy as np
from os import listdir
from os.path import join

car_types = ['hatch', 'sedan', 'suv']

def resize_image(img, size=(64,64)):

    h, w = img.shape[:2]

    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)

    dif = h if h > w else w

    interpolation = cv2.INTER_AREA if dif > (size[0]+size[1])//2 else cv2.INTER_CUBIC

    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    if len(img.shape) == 2:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        c = img.shape[2]
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]

    return cv2.resize(mask, size, interpolation)

spath, dpath = join('data', 'cars'), join('data', 'norm')

if not os.path.isdir(dpath):
    os.mkdir(dpath)

imgs = {}


for ct in car_types:
    sp, dp = join(spath, ct), join(dpath, ct)
    alen = 1024
    imgs[ct] = np.zeros((1024, 64, 64, 3))
    if not os.path.isdir(dp):
        os.mkdir(dp)
    tlen = 0
    for f in listdir(sp):
        sf, df = join(sp, f), join(dp, f)
        img = cv2.imread(sf)
        img = resize_image(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        tlen += 1
        if tlen > alen:
            imgs[ct] = np.append(imgs[ct], np.zeros((1024, 64, 64, 3)))
            alen += 1024
        imgs[ct][tlen-1] = img
    imgs[ct] = imgs[ct][:tlen]
    print(imgs[ct].shape)

(332, 64, 64, 3)
(410, 64, 64, 3)
(337, 64, 64, 3)


In [17]:
#%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

from tensorflow.keras import utils 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trainx, testx = imgs['hatch'][:300,:,:,:], imgs['hatch'][300:,:,:,:]

trainy, testy = np.zeros(trainx.shape[0], dtype=float), np.zeros(testx.shape[0], dtype=float)

trainx = np.append(trainx, imgs['sedan'][:375,:,:,:], axis=0)
testx = np.append(testx, imgs['sedan'][375:,:,:,:], axis=0)

ltrain, ltest = trainx.shape[0] - trainy.shape[0], testx.shape[0] - testy.shape[0]

trainy, testy = np.append(trainy, np.ones(ltrain, dtype=float)), np.append(testy, np.ones(ltest, dtype=float))


print(trainx.shape, "y", trainy.shape)
print(testx.shape, 'y', testy.shape)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(trainx)

trainX, trainY = trainx, trainy
testX, testY   = testx, testy

for i in range(0):
    iterate = datagen.flow(trainx, trainy, batch_size=len(trainx), shuffle=True)
    x, y = iterate.next()
    trainX, trainY = np.append(trainX, x, axis=0), np.append(trainY, y, axis=0)

for i in range(0):
    iterate = datagen.flow(testx, testy, batch_size=len(testx), shuffle=True)
    x, y = iterate.next()
    testX, testY = np.append(testX, x, axis=0), np.append(testY, y, axis=0)


print(trainX.shape, "y", trainY.shape)
print(testX.shape, "y", testY.shape)

trainx, trainy = trainX, trainY
testx, testy   = testX, testY

trainx = trainx.astype('float32') / 255
testx  = testx.astype('float32') / 255

trainx_mean = np.mean(trainx, axis=(0, 1, 2))
trainx_std  = np.std(trainx, axis=(0, 1, 2))

print(trainx_mean, trainx_std)

trainx -= trainx_mean
trainx /= trainx_std

testx -= trainx_mean
testx /= trainx_std

trainX, trainY = trainx, trainy #utils.to_binary(trainy)
testX,  testY  = testx,  testy  #utils.to_binary(testy)

min_pix, max_pix = trainX.min(), trainX.max()

print(min_pix, max_pix)
print(testX.min(), testX.max())

print(trainX.shape, trainY.shape)

1.14.0
(675, 64, 64, 3) y (675,)
(67, 64, 64, 3) y (67,)
(675, 64, 64, 3) y (675,)
(67, 64, 64, 3) y (67,)
[0.35301414 0.34561545 0.34175763] [0.3740168  0.37273017 0.37292084]
-0.9438457 1.7650994
-0.9438457 1.7650994
(675, 64, 64, 3) (675,)


In [18]:
print(testY)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [19]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, Activation, MaxPool2D
from tensorflow.keras.layers import add, Input, Dense, Flatten, GlobalAvgPool2D, GlobalAvgPool1D
from tensorflow.keras.initializers import zeros
from tensorflow.keras import regularizers

def ResConv(x, kernel=(3, 3), depth=32, maxpool=False):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if maxpool :
        x = MaxPool2D()(x)
    x = Conv2D(depth, kernel, padding='same', use_bias=False)(x)
    return x

def ResUnit(x, depth=32, maxpool=False):
    x = ResConv(x, depth=depth, maxpool=maxpool)
    x = ResConv(x, depth=depth)
    return x
    
def ResNetBlock(x, nunit, depth=32, maxpool=False):
    assert nunit > 0, "Ensure there are at least 1 unit in the ResNet Block"
    nunit -= 1
    if maxpool:
        xskip = Conv2D(depth, (1, 1), strides=2, use_bias=False)(x)
    else: 
        xskip = x
    x = add([ResUnit(x, depth=depth, maxpool=maxpool), xskip])
    if nunit >= 1:
        nunit -= 1
        for i in range(nunit):
            x = add([ResUnit(x, depth=depth), x])
        x = add([ResUnit(x, depth=depth), x])
    return x

# Returns latent vector of 32 bytes
def ResNet9(x):
    x = Conv2D(64, (7, 7), strides=2, padding='same', use_bias=False)(x)
    x = MaxPool2D((3, 3),  strides=2, padding='same')(x)
    
    nunits   = (2, 3, 2)
    maxpools = (False, True, True)
    depths   = (64, 32, 32)
    
    for i in range(3):
        x = ResNetBlock(x, nunits[i], depth=depths[i], maxpool=maxpools[i])
    x = GlobalAvgPool2D()(x)
    return x

#def D_init():
#    xin = Input(shape=(64, 64, 3))
#    x = ResNet9(xin)
#    x = Dense(1, use_bias=False, activation='sigmoid')(x)
#    return Model(xin, x)

def Q_init():
    xin = Input(shape=(64, 64, 3))
    x = ResNet9(xin)
    x = Dense(1, use_bias=False, activation='sigmoid')(x)
    return Model(xin, x)


def D_init():
    xin = Input(shape=(64, 64, 3))
    x = Conv2D(1,(64, 64), use_bias=False, kernel_regularizer=regularizers.l2(0.01))(xin) 
    x = Flatten()(x)
    x = Activation('sigmoid')(x)
    return Model(xin, x)
    

In [20]:
from tensorflow.keras import Model
from tensorflow.keras.layers import UpSampling2D, Reshape, Conv2DTranspose

def InvResConv(x, kernel=(3, 3), depth=32, upscale=False):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if upscale :
        x = UpSampling2D((2,2))(x)
    x = Conv2D(depth, kernel, padding='same', use_bias=False)(x)
    return x

def InvResUnit(x, depth=32, upscale=False):
    x = InvResConv(x, depth=depth)
    x = InvResConv(x, depth=depth, upscale=upscale)
    return x
    
def InvResNetBlock(x, nunit, depth=32, upscale=False):
    assert nunit > 0, "Ensure there are at least 1 unit in the ResNet Block"
    nunit -= 1
    x = InvResUnit(x, depth=depth, upscale=upscale)
    if nunit >= 1:
        nunit -= 1
        for i in range(nunit):
            x = InvResUnit(x, depth=depth)
        x = InvResUnit(x, depth=depth)
    return x


def InvResNet9(x, size=(4,4)):
    depths   = (32, 32, 64)
    upscales = (True, True, False)
    nunits   = (2, 3, 2)

    x = UpSampling2D(size)(x)
    for i in range(3):
        x = InvResNetBlock(x, nunits[i], depth=depths[i], upscale=upscales[i])
    x = UpSampling2D((2,2))(x)
    x = Conv2DTranspose(3, (7, 7), strides=2, padding='same', use_bias=False)(x)
    return x

In [21]:
def G_init():
    xin = Input(shape=(32,), name="Input")
    x = Reshape((1,1,32))(xin)
    x = InvResNet9(x)
    x = Activation('relu')(x)
    model = Model(xin, x)
    return model

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = 8 #int(np.sqrt(s / r))
            h = 8 #int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = 0.0 #np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

datagen = ImageDataGenerator(
        fill_mode = 'constant',
        cval=0,
        rotation_range=20,
        width_shift_range=8,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=8,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        preprocessing_function=get_random_eraser(v_l=min_pix, v_h=max_pix, pixel_level=False)
)
datagen.fit(trainX)

In [23]:
import matplotlib.pyplot as plt
%matplotlib inline

def accuracy(test_x, test_y, model):
    result = model.predict(test_x)
    predicted_class = np.argmax(result)
    true_class = np.argmax(test_y)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['bacc'])+1),model_history.history['bacc'])
    axs[0].plot(range(1,len(model_history.history['val_bacc'])+1),model_history.history['val_bacc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['bacc'])+1),len(model_history.history['bacc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [24]:
BATCH_SIZE = 64
WT_DECAY   = 1e-5
MOMENTUM   = 0.90
LEARNING_RATE = 0.002

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import categorical_crossentropy, binary_crossentropy
from tensorflow.keras import backend as K

def l2_weights(model):
    l2 = 0
    for layer in model.layers: 
        wt = layer.weights
        if len(wt) > 0:
            l2 += K.sum(K.pow(wt, 2))
    return l2

def reg_loss(model):
    def rloss(y_true, y_pred):
        return WT_DECAY*l2_weights(model)
    return rloss

def loss_with_regularization(model):
    def loss(y_true, y_pred):
        return binary_crossentropy(y_true, y_pred, True) + reg_loss(model)(y_true, y_pred)
    return loss

def bacc(y_true, y_pred):
    y_pred = K.cast(y_pred > 0.5, dtype=y_true.dtype)
    return K.mean(K.equal(y_true, y_pred), axis=-1)

Q = Q_init()
optimizer = SGD(lr=LEARNING_RATE, momentum=MOMENTUM, nesterov=True)
Q.compile(optimizer=optimizer, loss=loss_with_regularization(Q), metrics=[bacc, reg_loss(Q)])
Q.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 32, 32, 64)   9408        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 16, 16, 64)   0           conv2d_32[0][0]                  
__________________________________________________________________________________________________
batch_normalization_28 (BatchNo (None, 16, 16, 64)   256         max_pooling2d_3[0][0]            
____________________________________________________________________________________________

In [25]:
D = D_init()

D.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['acc'])
D.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 1, 1, 1)           12288     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1)                 0         
_________________________________________________________________
activation_44 (Activation)   (None, 1)                 0         
Total params: 12,288
Trainable params: 12,288
Non-trainable params: 0
_________________________________________________________________


In [26]:
G = G_init()

G.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 32)]              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 32)          0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_42 (Batc (None, 4, 4, 32)          128       
_________________________________________________________________
activation_45 (Activation)   (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 4, 4, 32)          9216      
_________________________________________________________________
batch_normalization_43 (Batc (None, 4, 4, 32)          128 

In [27]:
def gan_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred)*1e6

def define_gan(g_model, d_model, q_model):
    d_model.trainable = False
    q_model.trainable = False
    d_output = d_model(g_model.output)
    q_output = q_model(g_model.output)
    gan = Model(g_model.input, [d_output, q_output])
    opt = Adam(lr=0.0002, beta_1=0.5)
    gan.compile(loss=[gan_loss, 'binary_crossentropy'], optimizer=opt, metrics=['acc'])
    return gan

In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

mcp = ModelCheckpoint("q.hf5", monitor='val_bacc', save_weights_only=True, save_best_only=True)

for i in range(100):
    Q.fit_generator(datagen.flow(trainX, trainY, batch_size=BATCH_SIZE), epochs=100,  shuffle=True, 
                    validation_data=(testX, testY), callbacks=[mcp])
    
    Q.save_weights("qb.hf5")
    
    Q.load_weights("q.hf5")
    func = K.function(Q.input, Q.layers[-2].output)
    Gtrain, Gval = func(trainX), func(testX) 
    
    gan = define_gan(G, D, Q)
    Q.trainable = True
    
    gan.fit(Gtrain, [np.zeros((Gtrain.shape[0],)), trainY], epochs=100,
            validation_data=(Gval, [np.zeros((Gval.shape[0],)), testY]))
    
    fake = K.function(gan.input, G.output)(Gtrain)
    Dx = np.append(trainX, fake, axis=0)
    Dy = np.append(np.ones((trainX.shape[0],)), np.zeros((fake.shape[0],)))
    
    s = np.arange(Dx.shape[0])
    np.random.shuffle(s)
    Dx, Dy = Dx[s], Dy[s]
    D.fit(Dx, Dy, batch_size=BATCH_SIZE, validation_data=(testX, np.ones((testX.shape[0],))))

    Q.load_weights("qb.hf5")

Epoch 1/100
11/11 [==============================] - 1s 56ms/step - loss: 3.3150 - bacc: 0.9748 - rloss: 2.8200 - val_loss: 3.0968 - val_bacc: 0.5224 - val_rloss: 2.5120
Epoch 2/100
11/11 [==============================] - 1s 55ms/step - loss: 2.7873 - bacc: 0.9719 - rloss: 2.2901 - val_loss: 2.6060 - val_bacc: 0.5224 - val_rloss: 2.0422
Epoch 3/100
11/11 [==============================] - 1s 55ms/step - loss: 2.3570 - bacc: 0.9793 - rloss: 1.8633 - val_loss: 2.2269 - val_bacc: 0.5224 - val_rloss: 1.6636
Epoch 4/100
11/11 [==============================] - 1s 55ms/step - loss: 2.0129 - bacc: 0.9748 - rloss: 1.5195 - val_loss: 1.9216 - val_bacc: 0.5224 - val_rloss: 1.3582
Epoch 5/100
11/11 [==============================] - 1s 54ms/step - loss: 1.7363 - bacc: 0.9689 - rloss: 1.2419 - val_loss: 1.6751 - val_bacc: 0.5224 - val_rloss: 1.1118
Epoch 6/100
11/11 [==============================] - 1s 53ms/step - loss: 1.5079 - bacc: 0.9837 - rloss: 1.0178 - val_loss: 1.6829 - val_bacc: 0.5224 

Epoch 49/100
11/11 [==============================] - 1s 57ms/step - loss: 0.5480 - bacc: 0.9852 - rloss: 0.0594 - val_loss: 0.5644 - val_bacc: 0.6716 - val_rloss: 0.0594
Epoch 50/100
11/11 [==============================] - 1s 59ms/step - loss: 0.5534 - bacc: 0.9822 - rloss: 0.0594 - val_loss: 0.5799 - val_bacc: 0.6567 - val_rloss: 0.0593
Epoch 51/100
11/11 [==============================] - 1s 55ms/step - loss: 0.5491 - bacc: 0.9837 - rloss: 0.0592 - val_loss: 0.5776 - val_bacc: 0.6567 - val_rloss: 0.0593
Epoch 52/100
11/11 [==============================] - 1s 54ms/step - loss: 0.5507 - bacc: 0.9822 - rloss: 0.0594 - val_loss: 0.5732 - val_bacc: 0.6567 - val_rloss: 0.0595
Epoch 53/100
11/11 [==============================] - 1s 59ms/step - loss: 0.5533 - bacc: 0.9748 - rloss: 0.0594 - val_loss: 0.5681 - val_bacc: 0.6716 - val_rloss: 0.0594
Epoch 54/100
11/11 [==============================] - 1s 59ms/step - loss: 0.5493 - bacc: 0.9852 - rloss: 0.0593 - val_loss: 0.5509 - val_bacc: 0

Epoch 97/100
11/11 [==============================] - 1s 58ms/step - loss: 0.5527 - bacc: 0.9763 - rloss: 0.0602 - val_loss: 0.5348 - val_bacc: 0.7910 - val_rloss: 0.0602
Epoch 98/100
11/11 [==============================] - 1s 56ms/step - loss: 0.5516 - bacc: 0.9852 - rloss: 0.0602 - val_loss: 0.5513 - val_bacc: 0.6866 - val_rloss: 0.0602
Epoch 99/100
11/11 [==============================] - 1s 58ms/step - loss: 0.5514 - bacc: 0.9793 - rloss: 0.0602 - val_loss: 0.5666 - val_bacc: 0.6716 - val_rloss: 0.0604
Epoch 100/100
11/11 [==============================] - 1s 56ms/step - loss: 0.5504 - bacc: 0.9852 - rloss: 0.0605 - val_loss: 0.5630 - val_bacc: 0.7313 - val_rloss: 0.0606
Train on 675 samples, validate on 67 samples
Epoch 1/200
675/675 [==============================] - 5s 7ms/sample - loss: 0.0855 - model_7_loss: 1.8091e-05 - model_6_loss: 0.0680 - model_7_acc: 1.0000 - model_6_acc: 0.9778 - val_loss: 2.1295 - val_model_7_loss: 3.5025e-11 - val_model_6_loss: 2.5506 - val_model_7_a

675/675 [==============================] - 1s 950us/sample - loss: 0.0609 - model_7_loss: 8.9581e-06 - model_6_loss: 0.1095 - model_7_acc: 1.0000 - model_6_acc: 0.9881 - val_loss: 1.0530 - val_model_7_loss: 3.8688e-07 - val_model_6_loss: 0.7465 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7015
Epoch 26/200
675/675 [==============================] - 1s 971us/sample - loss: 0.0614 - model_7_loss: 1.4038e-04 - model_6_loss: 0.0452 - model_7_acc: 1.0000 - model_6_acc: 0.9867 - val_loss: 1.1376 - val_model_7_loss: 1.9245e-09 - val_model_6_loss: 0.7878 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.6716
Epoch 27/200
675/675 [==============================] - 1s 989us/sample - loss: 0.0425 - model_7_loss: 5.4677e-05 - model_6_loss: 0.0266 - model_7_acc: 1.0000 - model_6_acc: 0.9941 - val_loss: 1.3462 - val_model_7_loss: 1.7990e-07 - val_model_6_loss: 1.9559 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7015
Epoch 28/200
675/675 [==============================] - 1s 1ms/sample - loss: 0.063

675/675 [==============================] - 1s 959us/sample - loss: 0.0505 - model_7_loss: 3.6241e-05 - model_6_loss: 0.0348 - model_7_acc: 1.0000 - model_6_acc: 0.9896 - val_loss: 1.3429 - val_model_7_loss: 8.1486e-06 - val_model_6_loss: 1.3155 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7313
Epoch 52/200
675/675 [==============================] - 1s 967us/sample - loss: 0.0395 - model_7_loss: 4.4300e-05 - model_6_loss: 0.0239 - model_7_acc: 1.0000 - model_6_acc: 0.9941 - val_loss: 1.3682 - val_model_7_loss: 1.2003e-05 - val_model_6_loss: 0.9716 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7164
Epoch 53/200
675/675 [==============================] - 1s 959us/sample - loss: 0.0397 - model_7_loss: 1.0943e-04 - model_6_loss: 0.0238 - model_7_acc: 1.0000 - model_6_acc: 0.9926 - val_loss: 1.3650 - val_model_7_loss: 1.4750e-06 - val_model_6_loss: 1.0683 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7015
Epoch 54/200
675/675 [==============================] - 1s 970us/sample - loss: 0.0

675/675 [==============================] - 1s 963us/sample - loss: 0.0333 - model_7_loss: 2.4591e-05 - model_6_loss: 0.0179 - model_7_acc: 1.0000 - model_6_acc: 0.9941 - val_loss: 1.5189 - val_model_7_loss: 9.1758e-07 - val_model_6_loss: 2.2109 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7164
Epoch 78/200
675/675 [==============================] - 1s 975us/sample - loss: 0.0392 - model_7_loss: 4.8672e-05 - model_6_loss: 0.0411 - model_7_acc: 1.0000 - model_6_acc: 0.9852 - val_loss: 1.6416 - val_model_7_loss: 1.8747e-06 - val_model_6_loss: 1.8169 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.6866
Epoch 79/200
675/675 [==============================] - 1s 964us/sample - loss: 0.0465 - model_7_loss: 2.2244e-06 - model_6_loss: 0.0316 - model_7_acc: 1.0000 - model_6_acc: 0.9896 - val_loss: 1.5086 - val_model_7_loss: 2.8238e-05 - val_model_6_loss: 1.6510 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7015
Epoch 80/200
675/675 [==============================] - 1s 981us/sample - loss: 0.0

675/675 [==============================] - 1s 940us/sample - loss: 0.0530 - model_7_loss: 2.4268e-29 - model_6_loss: 0.0376 - model_7_acc: 1.0000 - model_6_acc: 0.9911 - val_loss: 1.2566 - val_model_7_loss: 1.5719e-23 - val_model_6_loss: 0.8691 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7015
Epoch 104/200
675/675 [==============================] - 1s 949us/sample - loss: 0.0482 - model_7_loss: 2.1628e-29 - model_6_loss: 0.0323 - model_7_acc: 1.0000 - model_6_acc: 0.9926 - val_loss: 1.3158 - val_model_7_loss: 8.6218e-24 - val_model_6_loss: 1.5289 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7015
Epoch 105/200
675/675 [==============================] - 1s 934us/sample - loss: 0.0476 - model_7_loss: 1.0789e-28 - model_6_loss: 0.0345 - model_7_acc: 1.0000 - model_6_acc: 0.9881 - val_loss: 1.2905 - val_model_7_loss: 4.0120e-24 - val_model_6_loss: 1.1111 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7015
Epoch 106/200
675/675 [==============================] - 1s 989us/sample - loss: 

675/675 [==============================] - 1s 941us/sample - loss: 0.0443 - model_7_loss: 1.1981e-26 - model_6_loss: 0.0823 - model_7_acc: 1.0000 - model_6_acc: 0.9926 - val_loss: 1.1826 - val_model_7_loss: 1.8512e-25 - val_model_6_loss: 1.6021 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7164
Epoch 130/200
675/675 [==============================] - 1s 960us/sample - loss: 0.0419 - model_7_loss: 4.3179e-27 - model_6_loss: 0.0263 - model_7_acc: 1.0000 - model_6_acc: 0.9941 - val_loss: 1.3833 - val_model_7_loss: 3.0057e-23 - val_model_6_loss: 1.3583 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.6866
Epoch 131/200
675/675 [==============================] - 1s 946us/sample - loss: 0.0541 - model_7_loss: 6.7071e-27 - model_6_loss: 0.1221 - model_7_acc: 1.0000 - model_6_acc: 0.9911 - val_loss: 1.4775 - val_model_7_loss: 1.5561e-24 - val_model_6_loss: 1.0250 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.6716
Epoch 132/200
675/675 [==============================] - 1s 984us/sample - loss: 

675/675 [==============================] - 1s 985us/sample - loss: 0.0306 - model_7_loss: 6.3603e-26 - model_6_loss: 0.0155 - model_7_acc: 1.0000 - model_6_acc: 0.9941 - val_loss: 1.7488 - val_model_7_loss: 2.2352e-29 - val_model_6_loss: 1.9353 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7164
Epoch 156/200
675/675 [==============================] - 1s 948us/sample - loss: 0.0372 - model_7_loss: 1.0246e-26 - model_6_loss: 0.0217 - model_7_acc: 1.0000 - model_6_acc: 0.9911 - val_loss: 1.7571 - val_model_7_loss: 2.9934e-29 - val_model_6_loss: 2.0192 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7164
Epoch 157/200
675/675 [==============================] - 1s 958us/sample - loss: 0.0506 - model_7_loss: 1.6165e-27 - model_6_loss: 0.0354 - model_7_acc: 1.0000 - model_6_acc: 0.9896 - val_loss: 1.5368 - val_model_7_loss: 1.6841e-28 - val_model_6_loss: 1.6998 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7164
Epoch 158/200
675/675 [==============================] - 1s 948us/sample - loss: 

675/675 [==============================] - 1s 997us/sample - loss: 0.0497 - model_7_loss: 1.4823e-24 - model_6_loss: 0.0337 - model_7_acc: 1.0000 - model_6_acc: 0.9941 - val_loss: 1.5219 - val_model_7_loss: 2.4322e-26 - val_model_6_loss: 1.0564 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7164
Epoch 182/200
675/675 [==============================] - 1s 1ms/sample - loss: 0.0297 - model_7_loss: 6.8942e-25 - model_6_loss: 0.0215 - model_7_acc: 1.0000 - model_6_acc: 0.9941 - val_loss: 1.6169 - val_model_7_loss: 6.4581e-27 - val_model_6_loss: 1.1187 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.6866
Epoch 183/200
675/675 [==============================] - 1s 977us/sample - loss: 0.0422 - model_7_loss: 5.9763e-26 - model_6_loss: 0.0408 - model_7_acc: 1.0000 - model_6_acc: 0.9926 - val_loss: 1.4818 - val_model_7_loss: 2.0008e-26 - val_model_6_loss: 1.0250 - val_model_7_acc: 1.0000 - val_model_6_acc: 0.7164
Epoch 184/200
675/675 [==============================] - 1s 988us/sample - loss: 0.